# Pre-canned Queries for Python
## Function to call Geography PermIDs
This query calls the Geography data model using the list of Geography PermIDs listed in Query Variables. The query returns all content items in the model:
 * AdminStatus
 * Identifier
 * Geography

 Additionally, three relationships are called:

 *CountryCapital* - which creates a new instance of the Geography
                     data model, and requests the name of the resulting
                     geograpghy PermID

 *CountryCurrency* - which creates a new instance of the Currency data
                     model, and uses a filter to return the Iso4217 currency code.

 *GeospatialChildren* - which creates a new instance of the Geography data model
                        and requests the names of the resulting children. This
                        relationship is also used demonstrate how to return an
                        array of the Geography PermIDs of the resulting children
                        without linking to a new instance of the Geography data model.


tags :     `AdminStatus`, `Identifier`, `Geography`, `Country`
## Import the library and load credentials
Credentials used by this notebook are stored in theconfig_files/credentials.ipynb. Please editcredentials.ipynb to set your credentials and run the next cell to continue with the scripts 

In [ ]:
from pathlib import Path 
import os
if Path.cwd().name != "config_files":
    p = Path("../config_files/") 
    os.chdir(p)

%run credentials.ipynb 
%run environment_selection.ipynb

To connect to the Refinitiv Data and opening a session,given function is used. To find out more about the library,you can visit [refinitiv-data library](https://pypi.org/project/refinitiv-data/)

In [ ]:
session = rdp.session.platform.Definition(
    app_key=APP_KEY,
    grant=rdp.session.platform.GrantPassword(
        username=RDP_LOGIN, password=RDP_PASSWORD
    ),
).get_session()
session.open()


## Graph call
Graph call with given sets of samples below. 

In [ ]:
%%writefile query_cache
query Geography($GeographyList: [String!]) {
    Geography(objectIds: $GeographyList) {
      AdminStatus {
        AdminStatus
        AdminStatusReason
        EffectiveFromDate
        EffectiveToDate
        EntityCreatedDate
        EntityModifiedDate
        GeographyId
        NotDeduplicatedFlag
        UnconfirmedFlag
      }
      Identifier {
        GeographyId
        # The filter applied here limits the identifier types to:
        #   320128 - Iso31661Numeric
        #   320099 - Iso31661Alpha2
        #   320135 - Iso31661Alpha3
        Identifiers(filter: {ValueTypeId: {IN: ["320128", "320099", "320135"]}}) {
          EffectiveFromDate
          EffectiveToDate
          PrimaryFlag
          ProviderId
          ProviderType
          SourceLinkUrl
          SourceType
          SourceUpdateDate
          TypeId
          Value
          ValueOrder
          ValueTypeCode
          ValueTypeId
        }
      }
      Geography {
        ActiveFromDate
        ActiveToDate
        ElevationNumber
        GeographyId
        LatitudeNumber
        LongName
        LongitudeNumber
        Name
        GeographyTypes {
          EffectiveFromDate
          EffectiveToDate
          TypeId
        }

        # The filter applied here limits the Names records returned to those where:
        #   The language of the name is 505074 - US English
        #   The type of name returned is 404500 - LongDescription
        Names(filter: {AND: [{LanguageId: {EQ: "505074"}}, {NameTypeId: {EQ: "404500"}}]}) {
          EffectiveFromDate
          EffectiveToDate
          LanguageId
          Name
          NameOrder
          NameTypeId
        }
      }

      # The CountryCapital relationship returns the Geography PermID of the capital city
      # of a country when the user provides a Geography PermID of a country. The relationship
      # creates a child inststance of the Geography data model, where the user can access all
      # attributes of the model.
      CountryCapital {
        Geography {
          LatitudeNumber
          LongitudeNumber
          Names(filter: {AND: [{LanguageId: {EQ: "505074"}}, {NameTypeId: {EQ: "404500"}}]}) {
            EffectiveFromDate
            EffectiveToDate
            LanguageId
            Name
            NameOrder
            NameTypeId
          }
        }
      }
        # The CountryCurrency relationship returns the Currency PermID of the currency of the
        # country represented by the Geography PermID being queried. The relationship links to the
        # Currency data model, where the user can access all attributes of the model. All fields
        # within CountryCurrency {..} are from the Currency data model.
      CountryCurrency {
        ObjectId
        Currency {
          DecimalPlaces
          Name
        }
        Identifier {
          # The filter applied here limits the type of identifier returned to 320001 - Iso4217

          Identifiers(filter: {ValueTypeId: {EQ: "320001"}}) {
            EffectiveFromDate
            EffectiveToDate
            PrimaryFlag
            ProviderId
            ProviderType
            SourceLinkUrl
            SourceType
            SourceUpdateDate
            TypeId
            Value
            ValueOrder
            ValueTypeCode
            ValueTypeId
          }
        }
      }

      # This relationship returns the Geography PermIDs of the objects that
      # are classed as children of the Geography PermID bewing queried. Where
      # a country Geography PermID is used, the children will be the states, counties
      # or prefectures (for example) of the country. The relationship creates a child
      # instance of the Geography model, where the user can access all attributes of
      # the model.
      # In this example the query simply returns the name of the child GeographyIDs
      GeospatialChildren {
        ObjectId
        Geography {
          Name
        }
      }
      # As an alternative to using GeospatialChildren, which creates a child instance of the
      # Geography model, users can simply request an array of the PermIDs that this relatiopnship
      # returns
      GeospatialChildrenId
    }
  }


## Sample :  Query calls the Geography data model using the list of Geography PermIDs

In [ ]:

variables = {'GeographyList': ['100148', '100025']}

In [ ]:
%run config_request_handler.ipynb

In [3]:
%run code_execution.ipynb

 
    data: { 
        Geography: [ 
            { 
                AdminStatus: { 
                    AdminStatus: Published, 
                    AdminStatusReason: null, 
                    EffectiveFromDate: 2010-03-26T12:06:25.000Z, 
                    EffectiveToDate: null, 
                    EntityCreatedDate: 2010-03-26T12:06:25.000Z, 
                    EntityModifiedDate: 2010-03-26T12:06:25.000Z, 
                    GeographyId: 100025, 
                    .................... 
......................................... 


## Close the default session when done

In [ ]:
session.close()